<a href="https://colab.research.google.com/github/NagamaniRayudu/NLP/blob/master/BERT_Extracting_Features_Practice_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


https://colab.research.google.com/drive/1RhmL0BqNe52FEbdSyLpkfVuCZxE7b5ke (Pre-trained BERT contextualized word embeddings with TPU)

https://github.com/google-research/bert/blob/master/extract_features.py

In [19]:
!pip install bert-tensorflow

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
import pprint

import bert
from bert import extract_features
from bert import modeling, tokenization

In [21]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(TPU_ADDRESS) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


In [0]:
#Now, load the pre-trained model of BERT.
pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 87
layer_indexes = [-1, -2, -3, -4]  
BATCH_SIZE = 32
OUTPUT_DIR = 'EMBEDDINGS'

In [0]:
def read_sequence(input_sentences):
  examples = []
  unique_id = 0
  for sentence in input_sentences:
    line = tokenization.convert_to_unicode(sentence)
    examples.append(extract_features.InputExample(unique_id=unique_id, text_a=line, text_b=None))
    unique_id += 1
  return examples
  

In [0]:
import collections
import numpy as np
def get_features(input_text, dim=768):

  bert_config = modeling.BertConfig.from_json_file(config_path)
  tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=True)

  examples =read_sequence(input_text)
  print(examples)
  features = extract_features.convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)
  
  unique_id_to_feature = {}
  for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

  #if TPU
  #is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  #tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
  #run_config = tf.contrib.tpu.RunConfig(
  #    cluster=tpu_cluster_resolver,
  #    tpu_config=tf.contrib.tpu.TPUConfig(
  #        num_shards=NUM_TPU_CORES,
  #        per_host_input_for_training=is_per_host))

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  run_config = tf.contrib.tpu.RunConfig(
      master=None,
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  model_fn = extract_features.model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=checkpoint_path,
      layer_indexes=layer_indexes,
      use_tpu=False,
      use_one_hot_embeddings=False)
  
  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=BATCH_SIZE, 
      train_batch_size=BATCH_SIZE)
  
  input_fn = extract_features.input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)
  
  for result in estimator.predict(input_fn, yield_single_examples=True):
    unique_id = int(result["unique_id"])
    feature = unique_id_to_feature[unique_id]
    all_features = []
    output = collections.OrderedDict()
    for (i, token) in enumerate(feature.tokens):
      all_layers = []
      for (j, layer_index) in enumerate(layer_indexes):
          layer_output = result["layer_output_%d" % j]
          layer_output_flat = np.array([x for x in layer_output[i:(i+1)].flat])
          all_layers.append(layer_output_flat)
      output[token]=sum(all_layers)[:dim]

  return output

In [25]:
embeddings = get_features(["This is a test"])
print(embeddings)

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] this is a test [SEP]
INFO:tensorflow:input_ids: 101 2023 2003 1037 3231 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpeki_lvrg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {